In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('/Users/kevinhan/babyRLDataProcessing/DevEvObject_2024-08-29.csv')
# delete rows that don't have any actions
df = df[~(df['object1_action_state_action'].isna() &
                    df['object2_action_state_action'].isna())]

print(pd.concat([df['object1_action_state_action'], df['object2_action_state_action']]).unique())


['p' 's' 'd' 't' 'pu' 'c' 'i' 'o' 'h' 'th' nan 'f']


In [3]:
agent1 = df[df['ID_subjsess'] == '15_10']
agent2 = df[df['ID_subjsess'] == '29_07']
agent3 = df[df['ID_subjsess'] == '34_11']

print(agent1.shape)
print(agent2.shape)
print(agent3.shape)

(138, 54)
(116, 54)
(44, 54)


In [4]:
unique_toys1 = pd.concat([agent1['object1_obj'], agent1['object2_obj']]).unique()
unique_toys2 = pd.concat([agent2['object1_obj'], agent2['object2_obj']]).unique()
unique_toys3 = pd.concat([agent3['object1_obj'], agent3['object2_obj']]).unique()
print(unique_toys1)
print(unique_toys2)
print(unique_toys3)


['r' 'f' 'cu' 'bu' nan 'rb' 'a' 'pb' 'p']
['rb' 'bu' nan 'sh' 'f' 'g' 'p' 't']
['r' 'rb' 's' 'sh' nan 't']


In [5]:
obj_states = {'r': [['noise', False]], # rattle
              'b': [['deform']], # broom
              'bs': [['deform'], ['detach'], ['reattach']], # broom set
              'm': [['noise']], # music toy alligator
              'a': [['noise', False]], # alligator busy box. Description seems odd, no states match it in data except noise
              'f': [['popup'], ['hidden']], # farm toy
              'g': [['detach'], ['reattach']], # gear toy
              'c': [['popup'], ['takeout']], # cabin with winnie
              's': [['putin']], # stroller
              'p': [['noise', False],['putin']], # pink pigs and coins (piggie bank?)
              'sh': [['noise', False]], # shapesorter with 3 shapes
              'ca': [['putin']], # cart
              't': [], # tree busy box, seems to be crossed out on document? No states match it in data either
              'pb': [['deform']], # pink beach ball
              'rb': [['deform']], # red spiky ball
              'y': [], # yellow donut ring. Nothing on document or data
              'bu': [['takeout'], ['putin']], # bucket with 6 balls
              'cu': [] # cubes with 6 colors. Nothing about states on document and no states match it in data
              }

# EDIT: Taking out deform state because actions are not defined for it yet and it's also not in data
for key, states in obj_states.items():
    obj_states[key] = [state for state in states if state != ['deform']]
for key, states in obj_states.items():
  states.append(['inhand', False])
  for state in states:
    if len(state) == 1:
      state.append('neutral')


# Output the updated obj_states dictionary
print(obj_states)

{'r': [['noise', False], ['inhand', False]], 'b': [['inhand', False]], 'bs': [['detach', 'neutral'], ['reattach', 'neutral'], ['inhand', False]], 'm': [['noise', 'neutral'], ['inhand', False]], 'a': [['noise', False], ['inhand', False]], 'f': [['popup', 'neutral'], ['hidden', 'neutral'], ['inhand', False]], 'g': [['detach', 'neutral'], ['reattach', 'neutral'], ['inhand', False]], 'c': [['popup', 'neutral'], ['takeout', 'neutral'], ['inhand', False]], 's': [['putin', 'neutral'], ['inhand', False]], 'p': [['noise', False], ['putin', 'neutral'], ['inhand', False]], 'sh': [['noise', False], ['inhand', False]], 'ca': [['putin', 'neutral'], ['inhand', False]], 't': [['inhand', False]], 'pb': [['inhand', False]], 'rb': [['inhand', False]], 'y': [['inhand', False]], 'bu': [['takeout', 'neutral'], ['putin', 'neutral'], ['inhand', False]], 'cu': [['inhand', False]]}


In [12]:
def agent2MDP(agentDF, obj_states):
  MDP = []
  total_state = obj_states
  step_count = 0
  prev_noise_state1, prev_noise_state2 = False, False
  for index, step in agentDF.iterrows():
    # Initialize previous state
    old_state = total_state

    # Check if noise state was activated in previous state. If so, turn it off (it will get updated below if noise action is continued)
    if prev_noise_state1 or prev_noise_state2:
      set_noise_state_false(total_state)
      
    # Update actions
    actions = []
    if isinstance(step['object1_action_state_action'], str):
      actions.append(step['object1_action_state_action'])
    if isinstance(step['object2_action_state_action'], str):
      actions.append(step['object2_action_state_action'])

    # Check if baby is still playing with same object(s)
    if not step_count == 0:
      if isinstance(last_obj1, str):
        if last_obj1 != step['object1_obj']:
          total_state[last_obj1][-1].pop()
          total_state[last_obj1][-1].append(False)
      if isinstance(last_obj2, str):
        if last_obj2 != step['object2_obj']:
          total_state[last_obj2][-1].pop()
          total_state[last_obj2][-1].append(False)
    last_obj1 = step['object1_obj']
    last_obj2 = step['object2_obj']

    # Go through row and update object states accordingly
    if isinstance(step['object1_obj'], str):
      for state in total_state[step['object1_obj']]:
        if state[0] != 'inhand':
          # Check if state is in row
          if step['object1_action_state_' + state[0]] == 'y':
            state.pop()
            state.append(True)
            # Check if noise state is activated
            if state[0] == 'noise':
              prev_noise_state1 = True
        elif state[0] == 'inhand':
          state.pop()
          state.append(True)
    if isinstance(step['object2_obj'], str):
      for state in total_state[step['object2_obj']]:
        if state[0] != 'inhand':
          # Check i fstate is in row
          if step['object2_action_state_' + state[0]] == 'y':
            state.pop()
            state.append(True)
            # Check if noise state is activated
            if state[0] == 'noise':
              prev_noise_state2 = True
        elif state[0] == 'inhand':
          state.pop()
          state.append(True)
    step_count += 1
    MDP.append([old_state, actions, total_state, step_count])
  return MDP

def set_noise_state_false(obj_states):
    for obj, states in obj_states.items():
        for state in states:
            if state[0] == 'noise':
                if len(state) == 1:
                    state.append(False)  # Add False if only 'noise' is present
                else:
                    state[1] = False  # Set to False if second element exists and is not 'False'


In [15]:
MDP1, MDP2, MDP3 = agent2MDP(agent1, obj_states), agent2MDP(agent2, obj_states), agent2MDP(agent3, obj_states)
all_MDPs = [MDP1, MDP2, MDP3]


In [16]:
import json
with open('ALL_MDPS.json', 'w') as f:
    json.dump(all_MDPs, f)


In [17]:
with open('ALL_MDPS.json', 'r') as f:
    loaded_data = json.load(f)